# Modular Arithmetic and Shor's Algorithm Implementation
## Laura Lewis

For this, we used references from the following sources:<br/>
https://arxiv.org/pdf/quant-ph/9508027.pdf<br/>
https://arxiv.org/pdf/quant-ph/0205095.pdf<br/>
https://qiskit.org/textbook/ch-algorithms/shor.html<br/>
https://qiskit.org/textbook/ch-algorithms/quantum-fourier-transform.html<br/>
https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html<br/>

Also, see the README for a bit more info.

In [2]:
import cirq
import numpy as np
import math
from fractions import Fraction

First, we have the QFT, which is already implemented in one of the Qiskit links above.

In [3]:
def qft(res):
    def swap_registers(res):
        n = len(res)
        for i in range(n // 2):
            yield cirq.SWAP(res[i], res[n - i - 1])
    def qft_rotations(res, n):
        if n == 0:
            return
        n -= 1
        yield cirq.H(res[n])
        for i in range(n):
            cp = cirq.ops.cphase(np.pi/2**(n-i))
            yield cp(res[i], res[n])
        yield qft_rotations(res, n)
   
    n = len(res)
    yield qft_rotations(res, n)
    yield swap_registers(res)

# def qft(res):
#     yield cirq.qft(*res)

In [4]:
# inverse QFT for n qubits
def iqft(res):
    inverse_circuit = cirq.inverse(qft(res))
    return inverse_circuit

Now, following Beauregard, we need to implement many quantum circuits for modular arithmetic, namely modular exponentiation. We follow her paper to implement all of the arithmetic operations there for a general modulus.

In [5]:
# adder circuit: maps |x> to |x + b mod N> for b, N classical integers

# helper for adding in phase first: x is a QuantumRegister and c, modulus are ints
# x is assumed to already by in the phase basis (i.e. we applied QFT to it already)
# this corresponds to \phi ADD(a) MOD(N) gate
# note that when calling this, it has to be called with another qubit (to store carry over)
def add_const_phase(x, b, modulus):
    n = len(x)
    mod_flag = cirq.LineQubit(3 * n)
    
    # |x + b>
    for i in range(n):
        power = 2 ** ((n - 1) - i)
        yield cirq.rz(b * np.pi / power)(x[i])
    
    # |x + b - N>
    for i in range(n - 1, -1, -1):
        power = 2 ** ((n - 1) - i)
        yield cirq.rz(-modulus * np.pi / power)(x[i])
    
    # copies MSB of |x + b - N> into mod_flag
    
    yield iqft(x)
    yield cirq.CNOT(x[n - 1], mod_flag)
    yield qft(x)
    
    # if x + b < modulus, modFlag is |1>, so add modulus back to register
    for i in range(n):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(modulus * np.pi / power).controlled()
        yield crz(mod_flag, x[i])
        
    # restore mod_flag to |0>
    for i in range(n - 1, -1, -1):
        power = 2 ** ((n - 1) - i)
        yield cirq.rz(-b * np.pi / power, x[i])
    yield cirq.x(mod_flag)
    yield iqft(x)
    yield cirq.CNOT(x[n - 1], mod_flag)
    yield qft(x)
    for i in range(n):
        power = 2 ** ((n - 1) - i)
        yield cirq.rz(b * np.pi / power, x[i])
        
# controlled version of previous function (unaware if Cirq does this automatically but couldn't find)
def cadd_const_phase(control, x, b, modulus):
    n = len(x)
    mod_flag = cirq.LineQubit(3 * n + 1)
    for i in range(n):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(b * np.pi / power).controlled()
        yield crz(control, x[i])

    for i in range(n - 1, -1, -1):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(-modulus * np.pi / power).controlled()
        yield crz(control, x[i])
    
    yield iqft(x) # depth added by not having these controlled, but doesn't affect correctness
    yield cirq.CCNOT(control, x[n - 1], mod_flag)
    yield qft(x)
    
    for i in range(n):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(modulus * np.pi / power).controlled()
        ccrz = crz.controlled()
        yield ccrz(control, mod_flag, x[i])
        
    for i in range(n - 1, -1, -1):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(-b * np.pi / power).controlled()
        yield crz(control, x[i])
    yield cirq.CNOT(control, mod_flag)
    yield iqft(x)
    yield cirq.CCNOT(control, x[n - 1], mod_flag)
    yield qft(x)
    for i in range(n):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(b * np.pi / power).controlled()
        yield crz(control, x[i])

# doubly controlled version of previous function (unaware if Cirq does this automatically but couldn't find)
def ccadd_const_phase(control1, control2, x, b, modulus):
    n = len(x)
    mod_flag = cirq.LineQubit(3 * n + 3)
    for i in range(n):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(b * np.pi / power).controlled()
        ccrz = crz.controlled()
        yield ccrz(control1, control2, x[i])

    for i in range(n - 1, -1, -1):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(-modulus * np.pi / power).controlled()
        ccrz = crz.controlled()
        yield ccrz(control1, control2, x[i])
    
    yield iqft(x) # depth added by not having these controlled, but doesn't affect correctness
    cccnot = cirq.CCNOT.controlled()
    yield cccnot(control1, control2, x[n - 1], mod_flag)
    yield qft(x)
    
    for i in range(n):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(modulus * np.pi / power).controlled()
        ccrz = crz.controlled()
        cccrz = ccrz.controlled()
        yield cccrz(control1, control2, mod_flag, x[i])
        
    for i in range(n - 1, -1, -1):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(-b * np.pi / power).controlled()
        ccrz = crz.controlled()
        yield ccrz(control1, control2, x[i])
    yield cirq.CCNOT(control1, control2, mod_flag)
    yield iqft(x)
    yield cccnot(control1, control2, x[n - 1], mod_flag)
    yield qft(x)
    for i in range(n):
        power = 2 ** ((n - 1) - i)
        crz = cirq.rz(b * np.pi / power).controlled()
        ccrz = crz.controlled()
        yield ccrz(control1, control2, x[i])
    
# adding constant in the computational basis (instead of in phase as before)
# not necessary for Shor's but useful
def add_const(x, b, modulus):
    n = len(x)
    anc = cirq.LineQubit(3 * n + 3)
    yield qft(x + [anc])
    yield add_const_phase(x + [anc], b, modulus)
    yield iqft(x + [anc]) 

Before continuing to the next step, just need to create a helper function following the extended Euclidean algorithm. This is a well-known algorithm, so we follow several descriptions of the implementation online, i.e. https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm. This will allow us to find the multiplicative inverse of an integer in the ring of integers modulo $N$ using techniques from abstract algebra.

In [6]:
def extended_euclid(a, b): 
    if (a == 0): 
        return (b, 0, 1)  
    gcd, x, y = extended_euclid(b % a, a)
    new_x = y - (b // a) * x
    new_y = x
    return (gcd, new_x, new_y)

Continuing with the arithmetic operations,

In [7]:
# multiplier and adder gate: maps |x>|y> to |x>|y + (ax) mod N> where a, N are classical integers
# this corresponds to to CMULT(a) MOD(N) without being controlled
def mult_add(x, y, a, modulus):
    n = len(x)
    anc = cirq.LineQubit(3 * n + 4)
    yield qft(y + [anc])
    for i in range(n):
        b = (((2 ** i) % modulus) * a) % modulus
        yield cadd_const_phase(x[i], y + [anc], b, modulus)
    yield iqft(y + [anc])

# controlled version of previous function
def cmult_add(control, x, y, a, modulus):
    n = len(x)
    anc = cirq.LineQubit(3 * n + 5)
    yield qft(y + [anc])
    for i in range(n):
        b = (((2 ** i) % modulus) * a) % modulus
        yield ccadd_const_phase(control, x[i], y + [anc], b, modulus)
    yield iqft(y + [anc])
        
# most important operation for Shor's
# multiplication by an integer: maps |x> to |(ax) mod N> where a, N are classical integers
def mult(x, a, modulus):
    n = len(x)
    
    # |x>|0>
    anc = cirq.LineQubit.range(3 * n + 6, 4 * n + 6)
    
    # |x>|(ax) mod N>
    yield mult_add(x, anc, a, modulus)
    
    # |(ax) mod N>|x>
    for i in range(n):
        yield cirq.SWAP(anc[i], x[i])
        
    # need to compute a^{-1} mod N so that we can uncompute |x> to |0>
    gcd, c, d = extended_euclid(a, modulus)
    assert(gcd == 1)
    a_inverse = c % modulus
    
    # now with this, we want to use mult_add to get |(ax) mod N>|0>
    # this is the same as if we subtract |x - a^{-1}(ax) mod N> = |x - x mod N> = |0> in the second register
    yield cirq.inverse(mult_add(x, anc, a_inverse, modulus))

# controlled version of previous function
def cmult(control, x, a, modulus):
    n = len(x)
    
    anc = cirq.LineQubit.range(4 * n + 7, 5 * n + 7)
    yield cmult_add(control, x, anc, a, modulus)
    for i in range(n):
        yield cirq.CSWAP(control, anc[i], x[i])
    gcd, c, d = extended_euclid(a, modulus)
    assert(gcd == 1)
    a_inverse = c % modulus
    yield cirq.inverse(cmult_add(control, x, anc, a_inverse, modulus))

Now, for the actual modular exponentiation circuits we wanted, where we use the approaches from https://arxiv.org/pdf/quant-ph/0205095.pdf and https://arxiv.org/pdf/quant-ph/9508027.pdf to implement this. 

In [8]:
# helper for modular exponentiation: maps |x> to |a^{2^j} x mod N> controlled on another qubit, where a, j, N are classical integers
def cmod_exp_helper(control, x, a, j, modulus):
    n = 2 ** j
    for i in range(n):
        yield cmult(control, x, a, modulus)

# modular exponentiation: maps |x>|y> to |x>|a^x * y mod N>, where a, N are classical integers
def mod_exp(x, y, a, modulus):
    n = len(x)
    for i in range(n):
        yield cmod_exp_helper(x[i], y, a, i, modulus)

Now, to actually implement the order finding algorithm, we have the following.

In [9]:
def find_order_sim(a, modulus, num_qubits):
    x = cirq.LineQubit.range(num_qubits)
    y = cirq.LineQubit.range(num_qubits, 2 * num_qubits)
    circuit = cirq.Circuit()
    
    circuit.append(cirq.X(y[0]))
    for i in range(num_qubits):
        circuit.append(cirq.H(x[i]))
    circuit.append(mod_exp(x, y, a, modulus))
    circuit.append(qft(x))
    circuit.append(cirq.measure(*x, key='res'))
    simulator = cirq.Simulator()
    result = simulator.run(circuit)
    return result

From here, we mainly follow the tutorial https://qiskit.org/textbook/ch-algorithms/shor.html as well as the procedure in https://arxiv.org/pdf/quant-ph/0205095.pdf (both of which are the same).

In [10]:
def find_order(a, N): # switching notation from modulus to N for convenience
    num_qubits = 2 * math.ceil(math.log2(N))
    result = find_order_sim(a, N, num_qubits)
    results = dict(result.histogram(key='res'))
    phase = max(results, key=results.get) / (2 ** num_qubits)
    return phase
    

In [11]:
def factor(N):
    if (N % 2 == 0):
        return (2, N // 2)
    a = np.random.randint(2, N)
    gcd = math.gcd(a, N)
    if (gcd > 1):
        return (a, N // a)
    order = Fraction(find_order(a, N)).limit_denominator(N).denominator
    if (order < N and order % 2 == 0):
        guess1 = math.gcd((pow(r, order // 2, N) - 1) % N, N)
        guess2 = math.gcd((pow(r, order // 2, N) + 1) % N, N)
        for guess in [guess1, guess2]:
            if (N % guess == 0 and guess != N and guess != 1):
                return (guess, N // guess)
    return (1, N) # not found

In [12]:
factor(4)

(2, 2)

In [13]:
factor(6)

(2, 3)

In [14]:
factor(8)

(2, 4)

In [23]:
factor(9)

(3, 3)

Note the 4, 6, 8 were sanity checks. The 9 took a long time to run.